## Determine Main Contributers to Electricity RRP

## Stage 1: Data Processing to produce Dataset of 30 min Average of 5min Interconnector Dispatch Data

In [ ]:
# Import libraries 
# I've included a handful of libraries here that might be useful
import glob
import os, zipfile, shutil
import pathlib
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import sys
import time
from IPython.display import display # Used to display multiple pandas tables in one cell
from datetime import datetime
from datetime import timedelta
from scipy import stats
from pprint import pprint

## Dispatch Data for Interconnector Information

Source Data is AEMO Dispatch Data from http://www.nemweb.com.au/REPORTS/ARCHIVE/DispatchIS_Reports/.
One month at a time was analysed for this project.

This file contains public 5 minute dispatch data by region. Data covers interconnector flows, constraints, 
regional reference price, demand, dispatchable generation, dispatchable load, and ancillary services data. 

In [ ]:
# Set the path to the data files - change this to where you store your files
from_dir = 'F:\\RMIT_MasterofAnalytics\\LEP_DataScience\\Assignments\\Datasets\\DISPATCHIS_Nov16'
to_dir = 'F:\\RMIT_MasterofAnalytics\\LEP_DataScience\\Assignments\\Datasets\\DISPATCHIS_Unzipped'
extension = ".zip"

os.chdir(from_dir) # change directory from working dir to dir with files

for item in os.listdir(from_dir): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(to_dir) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

In [ ]:
# Set the path to the data files - change this to where you store your files
from_dir = 'F:\\RMIT_MasterofAnalytics\\LEP_DataScience\\Assignments\\Datasets\\DISPATCHIS_Unzipped'
to_dir = 'F:\\RMIT_MasterofAnalytics\\LEP_DataScience\\Assignments\\Datasets\\DISPATCHIS_Unzipcsv'
extension = ".zip"

os.chdir(from_dir) # change directory from working dir to dir with files

for item in os.listdir(from_dir): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(to_dir) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

In [ ]:
analysisPath = 'F:/RMIT_MasterofAnalytics/LEP_DataScience/Assignments/Datasets/DISPATCHIS_Unzipcsv/'
count = len([f for f in os.listdir(analysisPath) if os.path.isfile(os.path.join(analysisPath, f))])
print(count)

In [ ]:
#Change this if necessary
numfiles = 0

In [ ]:
# Set the path to the data files - change this to where you store your files
from_dir = 'F:\\RMIT_MasterofAnalytics\\LEP_DataScience\\Assignments\\Datasets\\DISPATCHIS_Unzipcsv'
to_dir = 'F:\\RMIT_MasterofAnalytics\\LEP_DataScience\\Assignments\\Datasets\\DISPATCHIS_Processed'
extension = ".CSV"

os.chdir(from_dir) # change directory from working dir to dir with files
count=0
df_temp = pd.DataFrame()
for i in range(16,150):
    print(i)
    for file in os.listdir(from_dir): # loop through items in dir
        if file.endswith(extension): # check for ".zip" extension
            if (count<2000):
                if count % 100 == 0:
                    print(count)
                input_file = os.path.join(from_dir, file)
                dest_file = os.path.join(to_dir, file)
                # Read the csv files into dataframes
                df_input = pd.read_csv(input_file,sep=',',skiprows=i,nrows=6,error_bad_lines=False,warn_bad_lines=False)
                if (df_input.columns[0] == 'I') & (df_input.columns[2] == 'INTERCONNECTORRES'):
                    df_temp = pd.concat([df_temp, df_input], axis=0)
                    count = count + 1
                    # Move file once processed
                    shutil.move(input_file, dest_file)
            
numfiles = numfiles + 1
outputfile = 'F:/RMIT_MasterofAnalytics/LEP_DataScience/Assignments/Datasets/DISPATCHIS_DataFrame/DISPATCHIS_DataFrameNov16_' + str(numfiles) + '.csv'
df_temp.to_csv(outputfile,index=False)

In [ ]:
analysisPath = 'F:/RMIT_MasterofAnalytics/LEP_DataScience/Assignments/Datasets/DISPATCHIS_DataFrame/Nov16/'
df_final = pd.DataFrame()
for file in os.listdir(analysisPath):
    if file.endswith(".csv"):
        input_file = os.path.join(analysisPath, file)
        # Read the csv files into dataframes
        df_input = pd.read_csv(input_file,sep=',',skiprows=0,error_bad_lines=False,warn_bad_lines=False)
        df_final = pd.concat([df_final, df_input], axis=0)

In [ ]:
print(df_final.shape)
print(df_final.head(5))

In [ ]:
# Convert 'SETTLEMENTDATE' to datetime field
df_final['SETTLEMENTDATE'] = pd.to_datetime(df_final['SETTLEMENTDATE'], errors='coerce')
df_final = df_final.dropna(subset=['SETTLEMENTDATE'])

In [ ]:
# Subtract one minute to allow masking into 30 minute aggregates
df_final['SETTLEMENTDATE'] = df_final['SETTLEMENTDATE'] - timedelta(minutes=1)

In [ ]:
# Create Date and Time Fields
df_final['DATE'] = df_final['SETTLEMENTDATE'].dt.date
df_final['HOUR'] = df_final['SETTLEMENTDATE'].dt.hour
df_final['MINUTE'] = df_final['SETTLEMENTDATE'].dt.minute
print(df_final['MINUTE'].head(5))

In [ ]:
# Masks created to calculate 30minute averages
mask1 = (df_final['MINUTE'] >  0.0 ) & (df_final['MINUTE'] <=  30.0 )
mask2 = (df_final['MINUTE'] >  30.0 ) & (df_final['MINUTE'] <=  59.0 )

In [ ]:
# Create groups for 30 minute averaging
df_Group1 = df_final.loc[mask1]
df_Group2 = df_final.loc[mask2]
print(df_Group1.tail(5))
print(df_Group2.tail(5))

In [ ]:
# Remove Unnecessary columns
df_final.drop(['IMPORTGENCONID','DISPATCHINTERVAL','I','RUNNO','SETTLEMENTDATE','DISPATCH'], axis=1,inplace=True)
df_final.drop(['LOCALLY_CONSTRAINED_IMPORT','LOCAL_PRICE_ADJUSTMENT_IMPORT','LOCALLY_CONSTRAINED_EXPORT','LOCAL_PRICE_ADJUSTMENT_EXPORT','FCASIMPORTLIMIT','FCASEXPORTLIMIT'], axis=1,inplace=True)
df_final.drop(['EXPORTGENCONID','MARGINALLOSS','INTERCONNECTORRES','INTERVENTION','3'], axis=1,inplace=True)

In [ ]:
print(df_final.columns)

In [ ]:
# Create 30 minute Averages
df_30min_Group1 = df_final.loc[mask1].groupby(['INTERCONNECTORID','HOUR','DATE'],as_index=False)['MWFLOW'].mean()
df_30min_Group1['MINUTE'] = 30.0
print(df_30min_Group1.tail(5))

In [ ]:
# Create 30 minute Averages
df_30min_Group2 = df_final.loc[mask2].groupby(['INTERCONNECTORID','HOUR','DATE'],as_index=False)['MWFLOW'].mean()
df_30min_Group2['MINUTE'] = 59.0
print(df_30min_Group2.tail(5))

In [ ]:
# Combine 30 minute Averages into one dataset
df_INTER30min = pd.concat([df_30min_Group1, df_30min_Group2], axis=0)

In [ ]:
df_INTER30min['DATE'] = df_INTER30min['DATE'].astype('datetime64[ns]')

In [ ]:
# Recreate 'DATETIME' field
df_INTER30min['DATETIME'] = df_INTER30min['DATE'] + pd.to_timedelta(df_INTER30min['HOUR'], unit='h') + pd.to_timedelta(df_INTER30min['MINUTE'], unit='m')

In [ ]:
## Need to add one minute to times with minute = 59
mask = (df_INTER30min['MINUTE']==59.0)
df_INTER30min.loc[mask,'DATETIME']= df_INTER30min.loc[mask,'DATETIME'] + timedelta(minutes=1)

In [ ]:
df_INTER30min.drop(['DATE','HOUR','MINUTE'], axis=1,inplace=True)

In [ ]:
df_INTERoutput = df_INTER30min.pivot(index='DATETIME', columns='INTERCONNECTORID', values='MWFLOW')

In [ ]:
df_INTERoutput['DATETIME'] = df_INTERoutput.index

In [ ]:
print(df_INTERoutput.head(5))

In [ ]:
# Save Dataset 
outputfile='F:/RMIT_MasterofAnalytics/LEP_DataScience/Assignments/Datasets/DispatchIS_Nov16_30minAverage.csv'
df_INTERoutput.to_csv(outputfile,index=False)